<a href="https://colab.research.google.com/github/jeguns/EP7173/blob/main/Unidad%2006/Selecci%C3%B3n_de_atributos_en_modelos_de_regresi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalación de paquetes

In [1]:
pip install boruta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 4.5 MB/s eta 0:00:00


## Lectura de datos

El dataset Boston tiene su origen en un estudio realizado en la década de 1970 por el Departamento de Investigación y Desarrollo de la Comisión de Boston en colaboración con el economista David Harrison y el profesor de estadística del MIT, Daniel L. Rubinfeld.

El objetivo del dataset es predecir el valor medio de las casas en diferentes áreas de Boston basándose en diversas características socioeconómicas, geográficas y ambientales.

Variables contenidas:

CRIM - Tasa de criminalidad per cápita por ciudad.

ZN - Proporción de suelo residencial zonificado para lotes de más de 25,000 pies cuadrados.

INDUS - Proporción de acres comerciales no minoristas por ciudad.

CHAS - Variable indicadora de Charles River (1 si el área está al lado del río; 0 en caso contrario).

NOX - Concentración de óxidos de nitrógeno (partes por 10 millones).

RM - Número medio de habitaciones por vivienda.

AGE - Proporción de unidades ocupadas por propietarios construidas antes de 1940.

DIS - Distancia ponderada a cinco centros de empleo de Boston.

RAD - Índice de accesibilidad a carreteras radiales.

TAX - Tasa de impuesto a la propiedad por cada $10,000.

PTRATIO - Ratio de alumnos por maestro en la ciudad.

B - 1000(Bk - 0.63)^2  donde Bk es la proporción de personas de raza negra en la ciudad.

LSTAT - Porcentaje de población con bajo estatus socioeconómico.

MEDV - Valor medio de las viviendas ocupadas por propietarios en miles de dólares (variable objetivo).

In [2]:
import pandas as pd
import numpy as np
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
datos = pd.read_csv('housing.csv', delim_whitespace=True, names=column_names)
datos.head()

<ipython-input-2-f8f9d724ce0f>:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  datos = pd.read_csv('housing.csv', delim_whitespace=True, names=column_names)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [3]:
array = datos.values
X = array[:,0:13]
Y = array[:,13]

In [4]:
datos.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


# Filtros

## Selección por umbral de varianza

In [5]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest, SelectPercentile, SelectFpr, SelectFdr, SelectFwe

selector_var = VarianceThreshold(threshold = 0)
selector_var.fit_transform(X)

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [8]:
selected_columns = datos.drop('MEDV', axis=1).columns[selector_var.get_support()]
selected_columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')

In [9]:
df_selected = pd.DataFrame(selector_var.transform(X), columns=selected_columns)
df_selected.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


## Selección por correlación

In [10]:
from sklearn.feature_selection import r_regression, f_regression, mutual_info_regression

selector_cor = SelectKBest(score_func = r_regression, k=4).fit(X, Y)
selector_cor

SelectKBest(k=4, score_func=<function r_regression at 0x7d9141de7250>)

In [11]:
selector_cor.scores_ # CORRELACIONES ENTRE CADA X E Y

array([-0.38830461,  0.36044534, -0.48372516,  0.17526018, -0.42732077,
        0.69535995, -0.37695457,  0.24992873, -0.38162623, -0.46853593,
       -0.50778669,  0.33346082, -0.73766273])

In [12]:
selector_cor.n_features_in_ # NÚMERO INICIAL DE VARIABLES

13

In [13]:
selector_cor.get_support() # QUÉ VARIABLES SE SELECCIONAN

array([False,  True, False, False, False,  True, False,  True, False,
       False, False,  True, False])

In [14]:
datos.drop('MEDV', axis=1).columns[selector_cor.get_support()]

Index(['ZN', 'RM', 'DIS', 'B'], dtype='object')

Seleccionando las variables más asociadas (independientemente si la asociación es directa o inversa)

In [18]:
selected_indices = np.argsort(np.abs(selector_cor.scores_))[-4:]
selected_indices

array([ 2, 10,  5, 12])

In [19]:
selected_columns = datos.columns[selected_indices]
selected_columns

Index(['INDUS', 'PTRATIO', 'RM', 'LSTAT'], dtype='object')

In [20]:
df_selected = pd.DataFrame(X[:, selected_indices], columns = selected_columns)
df_selected.head()

,INDUS,PTRATIO,RM,LSTAT
0,2.31,15.3,6.575,4.98
1,7.07,17.8,6.421,9.14
2,7.07,17.8,7.185,4.03
3,2.18,18.7,6.998,2.94
4,2.18,18.7,7.147,5.33


## Selección por el estadístico F de ANOVA

In [22]:
selector_anova = SelectKBest(score_func=f_regression, k=4).fit(X, Y)
selector_anova.scores_ # F

array([ 89.48611476,  75.2576423 , 153.95488314,  15.97151242,
       112.59148028, 471.84673988,  83.47745922,  33.57957033,
        85.91427767, 141.76135658, 175.10554288,  63.05422911,
       601.61787111])

In [23]:
selector_anova.pvalues_ # PV

array([1.17398708e-19, 5.71358415e-17, 4.90025998e-31, 7.39062317e-05,
       7.06504159e-24, 2.48722887e-74, 1.56998221e-18, 1.20661173e-08,
       5.46593257e-19, 5.63773363e-29, 1.60950948e-34, 1.31811273e-14,
       5.08110339e-88])

In [24]:
selector_anova.n_features_in_

13

In [25]:
selector_anova.get_support()

array([False, False,  True, False, False,  True, False, False, False,
       False,  True, False,  True])

In [26]:
selected_columns = datos.drop('MEDV', axis=1).columns[selector_anova.get_support()]
df_selected = pd.DataFrame(selector_anova.transform(X), columns=selected_columns)
df_selected.head()

,INDUS,RM,PTRATIO,LSTAT
0,2.31,6.575,15.3,4.98
1,7.07,6.421,17.8,9.14
2,7.07,7.185,17.8,4.03
3,2.18,6.998,18.7,2.94
4,2.18,7.147,18.7,5.33


In [27]:
selector_anova_k = SelectKBest(score_func=f_regression, k=4).fit(X, Y)
selected_columns = datos.drop('MEDV', axis=1).columns[selector_anova_k.get_support()]
df_selected = pd.DataFrame(selector_anova_k.transform(X), columns=selected_columns)
df_selected.head()

,INDUS,RM,PTRATIO,LSTAT
0,2.31,6.575,15.3,4.98
1,7.07,6.421,17.8,9.14
2,7.07,7.185,17.8,4.03
3,2.18,6.998,18.7,2.94
4,2.18,7.147,18.7,5.33


In [31]:
selector_anova_perc = SelectPercentile(score_func=f_regression, percentile = 50).fit(X, Y)
selected_columns = datos.drop('MEDV', axis=1).columns[selector_anova_perc.get_support()]
df_selected = pd.DataFrame(selector_anova_perc.transform(X), columns=selected_columns)
df_selected.head()

,INDUS,NOX,RM,TAX,PTRATIO,LSTAT
0,2.31,0.538,6.575,296.0,15.3,4.98
1,7.07,0.469,6.421,242.0,17.8,9.14
2,7.07,0.469,7.185,242.0,17.8,4.03
3,2.18,0.458,6.998,222.0,18.7,2.94
4,2.18,0.458,7.147,222.0,18.7,5.33


In [32]:
selector_anova_fpr = SelectFpr(score_func=f_regression, alpha = 0.01).fit(X, Y)
selected_columns = datos.drop('MEDV', axis=1).columns[selector_anova_fpr.get_support()]
df_selected = pd.DataFrame(selector_anova_fpr.transform(X), columns=selected_columns)
df_selected.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [33]:
selector_anova_fdr = SelectFdr(score_func=f_regression, alpha = 0.01).fit(X, Y)
selected_columns = datos.drop('MEDV', axis=1).columns[selector_anova_fdr.get_support()]
df_selected = pd.DataFrame(selector_anova_fdr.transform(X), columns=selected_columns)
df_selected.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [34]:
selector_anova_fwe = SelectFwe(score_func=f_regression, alpha = 0.01).fit(X, Y)
selected_columns = datos.drop('MEDV', axis=1).columns[selector_anova_fwe.get_support()]
df_selected = pd.DataFrame(selector_anova_fwe.transform(X), columns=selected_columns)
df_selected.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


## Selección por el indicador de información mutua

In [35]:
selector_im = SelectKBest(score_func=lambda X, y: mutual_info_regression(X, Y, random_state=42), k=4).fit(X, Y)
selector_im.scores_

array([0.34463284, 0.17779873, 0.46852263, 0.02610416, 0.46397544,
       0.52778867, 0.31439287, 0.29752666, 0.2251369 , 0.35434658,
       0.44539121, 0.16329708, 0.6691506 ])

In [36]:
selector_im.get_support()

array([False, False,  True, False,  True,  True, False, False, False,
       False, False, False,  True])

In [37]:
selected_columns = datos.drop('MEDV', axis=1).columns[selector_im.get_support()]
df_selected = pd.DataFrame(selector_im.transform(X), columns=selected_columns)
df_selected.head()

,INDUS,NOX,RM,LSTAT
0,2.31,0.538,6.575,4.98
1,7.07,0.469,6.421,9.14
2,7.07,0.469,7.185,4.03
3,2.18,0.458,6.998,2.94
4,2.18,0.458,7.147,5.33


In [38]:
selector_im_k = SelectKBest(score_func=lambda X, y: mutual_info_regression(X, Y, random_state=42), k=4).fit(X, Y)
selected_columns = datos.drop('MEDV', axis=1).columns[selector_im_k.get_support()]
df_selected = pd.DataFrame(selector_im_k.transform(X), columns=selected_columns)
df_selected.head()

,INDUS,NOX,RM,LSTAT
0,2.31,0.538,6.575,4.98
1,7.07,0.469,6.421,9.14
2,7.07,0.469,7.185,4.03
3,2.18,0.458,6.998,2.94
4,2.18,0.458,7.147,5.33


In [39]:
selector_im_perc = SelectPercentile(score_func=lambda X, y: mutual_info_regression(X, Y, random_state=42), percentile=50).fit(X, Y)
selected_columns = datos.drop('MEDV', axis=1).columns[selector_im_perc.get_support()]
df_selected = pd.DataFrame(selector_im_perc.transform(X), columns=selected_columns)
df_selected.head()

,INDUS,NOX,RM,TAX,PTRATIO,LSTAT
0,2.31,0.538,6.575,296.0,15.3,4.98
1,7.07,0.469,6.421,242.0,17.8,9.14
2,7.07,0.469,7.185,242.0,17.8,4.03
3,2.18,0.458,6.998,222.0,18.7,2.94
4,2.18,0.458,7.147,222.0,18.7,5.33


# Wrappers

In [40]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [41]:
x = datos.drop('MEDV', axis=1)
y = datos['MEDV']

## Forward selection

In [42]:
from sklearn.feature_selection import SequentialFeatureSelector, RFE
selector_forward = SequentialFeatureSelector(model, n_features_to_select = 'auto', scoring = 'neg_mean_squared_error', direction='forward', tol = 1e-3).fit(x, y)

Revisar [aquí](https://scikit-learn.org/dev/modules/model_evaluation.html#scoring-parameter) los posibles valores para el argumento scoring

In [43]:
selector_forward.n_features_in_

13

In [44]:
selector_forward.feature_names_in_

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype=object)

In [45]:
selector_forward.get_support() # se selecciona o no la variable?

array([ True,  True,  True,  True, False,  True, False,  True, False,
       False,  True, False,  True])

In [46]:
selector_forward.get_feature_names_out() # cómo se llaman las variables seleccionaadas

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'RM', 'DIS', 'PTRATIO', 'LSTAT'],
      dtype=object)

In [47]:
selector_forward.transform(x) # array reducido solo con las variables seleccionadas

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 4.0900e+00, 1.5300e+01,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 4.9671e+00, 1.7800e+01,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 4.9671e+00, 1.7800e+01,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1675e+00, 2.1000e+01,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.3889e+00, 2.1000e+01,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.5050e+00, 2.1000e+01,
        7.8800e+00]])

In [48]:
df_selected = datos[selector_forward.get_feature_names_out()]
df_selected.head()

,CRIM,ZN,INDUS,CHAS,RM,DIS,PTRATIO,LSTAT
0,0.00632,18.0,2.31,0,6.575,4.0900,15.3,4.98
1,0.02731,0.0,7.07,0,6.421,4.9671,17.8,9.14
2,0.02729,0.0,7.07,0,7.185,4.9671,17.8,4.03
3,0.03237,0.0,2.18,0,6.998,6.0622,18.7,2.94
4,0.06905,0.0,2.18,0,7.147,6.0622,18.7,5.33


In [52]:
selector_forward = SequentialFeatureSelector(model, n_features_to_select = 'auto', scoring = 'neg_mean_squared_error', direction='forward', tol = 4).fit(x, y)
df_selected = datos[selector_forward.get_feature_names_out()]
df_selected.head()
# tol = 4 significa que si el mean_squared_error disminuye en 4 o más unidades, sigue. Si disminuye en menos de 4, para.

,PTRATIO,LSTAT
0,15.3,4.98
1,17.8,9.14
2,17.8,4.03
3,18.7,2.94
4,18.7,5.33


In [53]:
selector_forward = SequentialFeatureSelector(model, n_features_to_select = 'auto', scoring = 'neg_mean_squared_error', direction='forward', tol = 1e-1).fit(x, y)
df_selected = datos[selector_forward.get_feature_names_out()]
df_selected.head()

,CRIM,CHAS,RM,PTRATIO,LSTAT
0,0.00632,0,6.575,15.3,4.98
1,0.02731,0,6.421,17.8,9.14
2,0.02729,0,7.185,17.8,4.03
3,0.03237,0,6.998,18.7,2.94
4,0.06905,0,7.147,18.7,5.33


In [54]:
selector_forward = SequentialFeatureSelector(model, n_features_to_select = 'auto', scoring = 'neg_mean_squared_error', direction='forward', tol = 1e-3).fit(x, y)
df_selected = datos[selector_forward.get_feature_names_out()]
df_selected.head()

,CRIM,ZN,INDUS,CHAS,RM,DIS,PTRATIO,LSTAT
0,0.00632,18.0,2.31,0,6.575,4.0900,15.3,4.98
1,0.02731,0.0,7.07,0,6.421,4.9671,17.8,9.14
2,0.02729,0.0,7.07,0,7.185,4.9671,17.8,4.03
3,0.03237,0.0,2.18,0,6.998,6.0622,18.7,2.94
4,0.06905,0.0,2.18,0,7.147,6.0622,18.7,5.33


In [55]:
selector_forward = SequentialFeatureSelector(model, n_features_to_select = 'auto', scoring = 'neg_mean_squared_error', direction='forward').fit(x, y)
df_selected = datos[selector_forward.get_feature_names_out()]
df_selected.head()
# si no se coloca tol, selecciona el 50% de las características

,CRIM,CHAS,RM,DIS,PTRATIO,LSTAT
0,0.00632,0,6.575,4.0900,15.3,4.98
1,0.02731,0,6.421,4.9671,17.8,9.14
2,0.02729,0,7.185,4.9671,17.8,4.03
3,0.03237,0,6.998,6.0622,18.7,2.94
4,0.06905,0,7.147,6.0622,18.7,5.33


## Backward selection

In [56]:
selector_backward = SequentialFeatureSelector(model, n_features_to_select = 'auto', scoring = 'neg_mean_squared_error', direction='backward', tol = -8e-1).fit(x, y)
df_selected = datos[selector_backward.get_feature_names_out()]
df_selected.head()

,ZN,NOX,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,18.0,0.538,4.0900,1,296.0,15.3,396.90,4.98
1,0.0,0.469,4.9671,2,242.0,17.8,396.90,9.14
2,0.0,0.469,4.9671,2,242.0,17.8,392.83,4.03
3,0.0,0.458,6.0622,3,222.0,18.7,394.63,2.94
4,0.0,0.458,6.0622,3,222.0,18.7,396.90,5.33


In [57]:
selector_backward = SequentialFeatureSelector(model, n_features_to_select = 'auto', scoring = 'neg_mean_squared_error', direction='backward', tol = -1e-1).fit(x, y)
df_selected = datos[selector_backward.get_feature_names_out()]
df_selected.head()

,CRIM,ZN,NOX,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,0.538,65.2,4.0900,1,296.0,15.3,396.90,4.98
1,0.02731,0.0,0.469,78.9,4.9671,2,242.0,17.8,396.90,9.14
2,0.02729,0.0,0.469,61.1,4.9671,2,242.0,17.8,392.83,4.03
3,0.03237,0.0,0.458,45.8,6.0622,3,222.0,18.7,394.63,2.94
4,0.06905,0.0,0.458,54.2,6.0622,3,222.0,18.7,396.90,5.33


In [58]:
selector_backward = SequentialFeatureSelector(model, n_features_to_select = 'auto', scoring = 'neg_mean_squared_error', direction='backward').fit(x, y)
df_selected = datos[selector_backward.get_feature_names_out()]
df_selected.head()
# si no se coloca tolerancia, asume el 50% del total de variables

,ZN,NOX,DIS,RAD,TAX,PTRATIO,LSTAT
0,18.0,0.538,4.0900,1,296.0,15.3,4.98
1,0.0,0.469,4.9671,2,242.0,17.8,9.14
2,0.0,0.469,4.9671,2,242.0,17.8,4.03
3,0.0,0.458,6.0622,3,222.0,18.7,2.94
4,0.0,0.458,6.0622,3,222.0,18.7,5.33


## Recursive Feature Elimination (RFE)

In [59]:
selector_rfe = RFE(model, n_features_to_select= 3).fit(x, y)

In [60]:
selector_rfe.ranking_

array([ 6,  8,  7,  1,  1,  1, 11,  3,  5,  9,  2, 10,  4])

In [61]:
selector_rfe.get_support()

array([False, False, False,  True,  True,  True, False, False, False,
       False, False, False, False])

In [62]:
selector_rfe.get_feature_names_out()

array(['CHAS', 'NOX', 'RM'], dtype=object)

In [63]:
df_selected = datos[selector_rfe.get_feature_names_out()]
df_selected.head()

,CHAS,NOX,RM
0,0,0.538,6.575
1,0,0.469,6.421
2,0,0.469,7.185
3,0,0.458,6.998
4,0,0.458,7.147


In [64]:
selector_rfe = RFE(model, n_features_to_select = 0.7).fit(x, y)
df_selected = datos[selector_rfe.get_feature_names_out()]
df_selected.head()

,CRIM,INDUS,CHAS,NOX,RM,DIS,RAD,PTRATIO,LSTAT
0,0.00632,2.31,0,0.538,6.575,4.0900,1,15.3,4.98
1,0.02731,7.07,0,0.469,6.421,4.9671,2,17.8,9.14
2,0.02729,7.07,0,0.469,7.185,4.9671,2,17.8,4.03
3,0.03237,2.18,0,0.458,6.998,6.0622,3,18.7,2.94
4,0.06905,2.18,0,0.458,7.147,6.0622,3,18.7,5.33


In [66]:
selector_rfe = RFE(model).fit(x, y)
df_selected = datos[selector_rfe.get_feature_names_out()]
df_selected.head()
# si no se indica la cantidad de variables a seleccionar, por defecto extrae la mitad

,CHAS,NOX,RM,DIS,PTRATIO,LSTAT
0,0,0.538,6.575,4.0900,15.3,4.98
1,0,0.469,6.421,4.9671,17.8,9.14
2,0,0.469,7.185,4.9671,17.8,4.03
3,0,0.458,6.998,6.0622,18.7,2.94
4,0,0.458,7.147,6.0622,18.7,5.33


## Exhaustive Feature Selection

La cantidad de modelos a ejecutar puede ser muy alta. En este caso, al tener 13 variables, son 13 modelos de 1 variable, 78 modelos de 2 variables, 286 modelos de 3 variables y 715 de 4 variables.

In [70]:
from math import comb
comb(13,1) + comb(13,2) + comb(13,3) + comb(13,4)

1092

In [73]:
comb(13,2), comb(13,3), comb(13,4)

(78, 286, 715)

In [74]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector
selector_efs = ExhaustiveFeatureSelector(model,
                                         min_features=1,
                                         max_features=4,
                                         scoring='neg_mean_squared_error',
                                         print_progress=True,
                                         cv=5)

In [75]:
import warnings
warnings.filterwarnings('ignore')
selector_efs.fit(x,y)

Features: 1092/1092

ExhaustiveFeatureSelector(estimator=LinearRegression(),
                          feature_groups=[[0], [1], [2], [3], [4], [5], [6],
                                          [7], [8], [9], [10], [11], [12]],
                          max_features=4, scoring='neg_mean_squared_error')

In [76]:
warnings.filterwarnings('default')

In [77]:
selector_efs.subsets_[0]

{'feature_idx': (0,),
 'cv_scores': array([ -37.29856463,  -90.26649703, -136.57347572,  -77.11581342,
         -51.62475583]),
 'avg_score': -78.57582132718443,
 'feature_names': ('CRIM',)}

In [78]:
selector_efs.subsets_[20]

{'feature_idx': (0, 8),
 'cv_scores': array([ -43.22058742,  -88.90886573, -129.71442608,  -81.22467121,
         -43.13274857]),
 'avg_score': -77.24025980277327,
 'feature_names': ('CRIM', 'RAD')}

In [79]:
selector_efs.subsets_[1091]

{'feature_idx': (9, 10, 11, 12),
 'cv_scores': array([-18.58824353, -41.34608947, -57.33376247, -51.61188811,
        -19.81583564]),
 'avg_score': -37.73916384369623,
 'feature_names': ('TAX', 'PTRATIO', 'B', 'LSTAT')}

In [80]:
selector_efs.best_score_

-33.462988003167666

In [81]:
selector_efs.best_idx_

(4, 7, 10, 12)

In [82]:
selected_features = selector_efs.best_feature_names_
selected_features

('NOX', 'DIS', 'PTRATIO', 'LSTAT')

In [83]:
df_selected = datos[list(selected_features)]
df_selected.head()

,NOX,DIS,PTRATIO,LSTAT
0,0.538,4.0900,15.3,4.98
1,0.469,4.9671,17.8,9.14
2,0.469,4.9671,17.8,4.03
3,0.458,6.0622,18.7,2.94
4,0.458,6.0622,18.7,5.33


Otros modelos:

In [84]:
from sklearn.svm import SVR
model_svr = SVR(kernel='linear')
selector_forward = SequentialFeatureSelector(model_svr, n_features_to_select = 'auto', direction='forward', tol = 1e-5).fit(x, y)
df_selected = datos[selector_forward.get_feature_names_out()]
df_selected.head()

,CRIM,ZN,INDUS,CHAS,NOX,DIS,RAD,PTRATIO,LSTAT
0,0.00632,18.0,2.31,0,0.538,4.0900,1,15.3,4.98
1,0.02731,0.0,7.07,0,0.469,4.9671,2,17.8,9.14
2,0.02729,0.0,7.07,0,0.469,4.9671,2,17.8,4.03
3,0.03237,0.0,2.18,0,0.458,6.0622,3,18.7,2.94
4,0.06905,0.0,2.18,0,0.458,6.0622,3,18.7,5.33


In [85]:
from sklearn.neighbors import KNeighborsRegressor
model_knr = KNeighborsRegressor(n_neighbors=5)
selector_forward = SequentialFeatureSelector(model_knr, n_features_to_select = 'auto', direction='forward', tol = 1e-5).fit(x, y)
df_selected = datos[selector_forward.get_feature_names_out()]
df_selected.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,CRIM,CHAS,RM,LSTAT
0,0.00632,0,6.575,4.98
1,0.02731,0,6.421,9.14
2,0.02729,0,7.185,4.03
3,0.03237,0,6.998,2.94
4,0.06905,0,7.147,5.33


## Boruta

In [86]:
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy

<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


In [87]:
x = datos.drop('MEDV', axis=1)
y = datos['MEDV']

rf = RandomForestRegressor(n_estimators=100)

In [92]:
boruta_selector = BorutaPy(estimator=rf, n_estimators='auto', verbose=0, random_state=2024) # random_state = semilla

In [93]:
boruta_selector.fit(x.values, y.values)

BorutaPy(estimator=RandomForestRegressor(n_estimators=42,
                                         random_state=RandomState(MT19937) at 0x7D9141320840),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7D9141320840)

In [94]:
boruta_selector.support_

array([ True, False, False, False,  True,  True,  True,  True, False,
        True,  True,  True,  True])

In [95]:
boruta_selector.ranking_

array([1, 4, 2, 5, 1, 1, 1, 1, 3, 1, 1, 1, 1])

In [96]:
selected_features = x.columns[boruta_selector.support_].tolist()
selected_features

['CRIM', 'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']

In [97]:
datos[selected_features + ['MEDV']]

,CRIM,NOX,RM,AGE,DIS,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,0.538,6.575,65.2,4.0900,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.469,6.421,78.9,4.9671,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.469,7.185,61.1,4.9671,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.458,6.998,45.8,6.0622,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.458,7.147,54.2,6.0622,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.573,6.593,69.1,2.4786,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.573,6.120,76.7,2.2875,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.573,6.976,91.0,2.1675,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.573,6.794,89.3,2.3889,273.0,21.0,393.45,6.48,22.0
